In [ ]:
#pipeline pseudocode
init_pipeline(params)
    #prepare_image_and_dirs - at now we manually put fits files in current dir
    init_astromatic_wrapper # create kwargs dicts for astromatic objects from global params

# assume we have some images image1.fits image2.fits image3.fits and corresponding to them bias dark and flat fits
# 1 pre-reduction of FITS (bias, dark ,flat - HOW???) output - pr_images
# 2 calibrate images using SCAMP (which uses SExtractor catalog)
    # create catalog from pre-reduced fits
    sextractor1 = aw.api.Astromatic(**sex_kwargs_1)
    sextractor1.run(pr_image1.fits) # output - sex_output_pr_1
    # send catalog to SCAMP - it will calculate astrometry and photometry solution (write into header)
    scamp = aw.api.Astromatic(**scamp_kwargs)
    scamp.run(sex_output_pr_1.fits) # output - calibrated_pr_1
    # for 2 and 3 the same way
# 3 co-addition and resampling of calibrated pre-reduced images
    swarp = aw.api.Astromatic(**swarp_kwargs)
    swarp.run(calibrated_pr_images) # output - preprocessed_coadd
# 4 extracting sources from preprocessed images
    sextractor2 = aw.api.Astromatic(**sex_kwargs_2)
    sextractor2.run(preprocessed_coadd.fits) # output - catalog_1
    psfex = aw.api.Astromatic(**psfex_kwargs)
    psfex.run(catalog_1) # output - psf_coadd
    sextractor3 = aw.api.Astromatic(**sex_kwargs_3)
    sextractor3.run(preprocessed_coadd.fits, psf_coadd.fits) # output - catalog_2
# catalog_2 - catalog of objects extracted from pre-reducted, preprocessed and co-added and psf-modeled fits
# futher processing:
    # a) put catalog into DB
    # b) add single images processing (not only co-added) - add to catalog objects wchich dissapeared after co-addition

# First, we check workability of components of astromatic software through astromatic_wrapper api

## Prepare invironment

In [1]:
import os
import datetime
import fnmatch
import re
import astromatic_wrapper as aw
from astropy.io import fits

In [2]:
aw.api.codes

{'PSFEx': 'psfex', 'SCAMP': 'scamp', 'SExtractor': 'sex', 'SWarp': 'swarp'}

In [3]:
# define mode and log path
mode = 'test'
wrk_dir = ''
log_path = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S') + '_' + mode

# set paths to work dirs
base_path = os.getcwd() if wrk_dir is None or not len(wrk_dir) else wrk_dir
paths = {
    'temp': os.path.join(base_path, 'temp'),
    #'log': os.path.join(base_path, 'log', log_path),
    'config': os.path.join(base_path, 'config'),
    'catalogs': os.path.join(base_path, 'catalogs'),
    'stacks': os.path.join(base_path, 'stacks'),
    'images': os.path.join(base_path, 'images')
        }
# build paths is needed for pipeline (it`s a code issue and will be elimenated in future)
build_paths = {}

# check if paths are ok
for val in paths.values():
    print val

/home/ser/Dev/Notebooks/test_pipeline/images
/home/ser/Dev/Notebooks/test_pipeline/catalogs
/home/ser/Dev/Notebooks/test_pipeline/config
/home/ser/Dev/Notebooks/test_pipeline/stacks
/home/ser/Dev/Notebooks/test_pipeline/temp


In [28]:
# set image filename
files = {'image': os.path.join(paths['images'], 'GRB130427_R60_001_001.fit')}

In [29]:
# prepare FITS file header for processing
hdulist = fits.open(files['image'])
hdulist[0].header

SIMPLE  =                    T / file does conform to FITS standard             
BITPIX  =                   16 / number of bits per data pixel                  
NAXIS   =                    2 / number of data axes                            
NAXIS1  =                  512 / length of data axis 1                          
NAXIS2  =                  512 / length of data axis 2                          
EXTEND  =                    T / FITS dataset may contain extensions            
COMMENT   FITS (Flexible Image Transport System) format is defined in 'Astronomy
COMMENT   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H 
BZERO   =                32768 / offset data range to that of unsigned short    
BSCALE  =                    1 / default scaling factor                         
DATE-OBS= '2013-05-14T14:24:34.7150' / Exposure start date/time (UTC)           
EXPTIME =                  60. / [s] Integration time                           
SHUTTER = 'AUTO    '        

### 1) SExtractor

In [30]:
# set params for sextractor execution
# set code type according to sextractor
sex_kwargs_1 = {'code': 'SExtractor'}
# set path to config file
sex_kwargs_1['config_file'] = os.path.join(paths['config'], 'default.sex')
# change some parameters in config (will not change config file set above)
sex_kwargs_1['config'] = {'CATALOG_NAME': os.path.join(paths['temp'], 
                                                os.path.basename(files['image']).replace('.fit', '.cat'))
                    }

sex_kwargs_1['config']['CATALOG_TYPE'] = 'FITS_LDAC'
sex_kwargs_1['config']['FILTER'] = 'N'
sex_kwargs_1['temp_path'] = paths['temp']
sex_kwargs_1['params'] = ['NUMBER', 'EXT_NUMBER', 'XWIN_IMAGE', 'YWIN_IMAGE', 'AWIN_IMAGE', 'BWIN_IMAGE',
                          'ERRAWIN_IMAGE','ERRBWIN_IMAGE', 'ERRTHETAWIN_IMAGE', 'ERRA_WORLD', 'ERRB_WORLD', 
                          'ERRTHETA_WORLD', 'X_WORLD', 'Y_WORLD', 'XWIN_WORLD', 'YWIN_WORLD', 
                          'FLUX_AUTO', 'FLUX_MAX', 'MAG_AUTO', 'FLUXERR_AUTO', 
                          # flags not supported yet (need Weightwatcher)
                          #'IMAFLAGS_ISO', 'FLAGS_WEIGHT',
                          'FLAGS', 'FLUX_RADIUS', 'ELONGATION']

# minimum catalog columns needed for SCAMP:
#XWIN_IMAGE
#YWIN_IMAGE
#ERRAWIN_IMAGE
#ERRBWIN_IMAGE
#ERRTHETAWIN_IMAGE
#ERRA_WORLD
#ERRB_WORLD
#ERRTHETA_WORLD
#X_WORLD
#Y_WORLD
#FLUX_RADIUS
#FLUX_AUTO
#FLUXERR_AUTO
#FLUX_MAX
#MAG_AUTO
#ELONGATION
#FLAGS
#AWIN_IMAGE
#BWIN_IMAGE

In [31]:
sextractor = aw.api.Astromatic(**sex_kwargs_1)

In [32]:
this_cmd, kwargs2 = sextractor.build_cmd(files['image'], **sex_kwargs_1)
print this_cmd

sex /home/ser/Dev/Notebooks/test_pipeline/images/GRB130427_R60_001_001.fit -c /home/ser/Dev/Notebooks/test_pipeline/config/default.sex -FILTER N -PARAMETERS_NAME /home/ser/Dev/Notebooks/test_pipeline/temp/sex.param -CATALOG_TYPE FITS_LDAC -CATALOG_NAME /home/ser/Dev/Notebooks/test_pipeline/temp/GRB130427_R60_001_001.cat


In [33]:
sextractor.run(files['image']) 

{'status': 'success'}

In [34]:
catalog = aw.utils.ldac.get_table_from_ldac(sex_kwargs_1['config']['CATALOG_NAME'])
catalog

NUMBER,EXT_NUMBER,XWIN_IMAGE,YWIN_IMAGE,AWIN_IMAGE,BWIN_IMAGE,ERRAWIN_IMAGE,ERRBWIN_IMAGE,ERRTHETAWIN_IMAGE,ERRA_WORLD,ERRB_WORLD,ERRTHETA_WORLD,X_WORLD,Y_WORLD,XWIN_WORLD,YWIN_WORLD,FLUX_AUTO,FLUX_MAX,MAG_AUTO,FLUXERR_AUTO,FLAGS,FLUX_RADIUS,ELONGATION
,,pix,pix,pix,pix,pix,pix,deg,deg,deg,deg,deg,deg,deg,deg,ct,ct,mag,ct,,pix,
int32,int16,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,float32,float32,float32,float32,int16,float32,float32
1,1,74.0506797733,4.96785582996,0.323244,0.289622,0.0185418,0.0184054,44.8963,5.40008e-06,3.77398e-06,-37.5049,173.19748825,27.6328150289,173.197495356,27.6328720298,1874.71,1444.24,-8.18233,74.1216,0,0.485663,1.68992
2,1,435.752504902,4.12222775746,0.506723,0.249457,0.0234493,0.0126799,-47.5847,6.25309e-06,3.38137e-06,47.5983,173.088704362,27.6326310121,173.088623675,27.6326436152,1790.76,970.529,-8.13259,84.6418,0,0.57194,2.0313
3,1,391.664085704,3.65109619705,1.30106,0.288675,0.0926271,0.022354,45.0,2.47005e-05,5.96106e-06,-44.9895,173.101805007,27.6323387867,173.101894252,27.6325228849,287.597,396.692,-6.14696,153.999,16,0.511613,4.50702
4,1,112.09876803,3.92879817309,0.787976,0.214852,0.0362528,0.00797585,61.7319,9.66705e-06,2.12845e-06,-61.744,173.186624908,27.6325439927,173.186042832,27.6325986426,-67.0791,866.306,99.0,116.252,16,0.0,3.66752
5,1,108.454015622,9.39888210087,0.727711,0.186901,0.082488,0.0217069,-27.1206,2.1996e-05,5.79152e-06,27.1079,173.187631956,27.6334934591,173.187140435,27.6340570144,772.069,492.583,-7.21914,83.2629,0,0.708332,3.89356
6,1,171.016293694,10.2398484152,1.17612,0.174485,0.0296752,0.00805462,-38.7114,7.91331e-06,2.14814e-06,38.7062,173.168327636,27.6331851572,173.168309107,27.634285449,2847.8,1274.26,-8.63627,95.3613,16,1.23039,6.74054
7,1,304.262985612,9.77950602873,0.72701,0.305383,0.0565512,0.00944447,-31.1592,1.50805e-05,2.51715e-06,31.1644,173.128213415,27.6341834105,173.128201511,27.6341631122,1519.67,1424.83,-7.95437,124.696,0,0.465004,2.38065
8,1,431.210482331,8.52234249694,1.09986,0.419533,0.0884621,0.030418,4.72838,2.35895e-05,8.11258e-06,-4.70234,173.090049909,27.6340057375,173.089990204,27.6338175391,993.887,541.043,-7.49334,179.785,0,1.08155,2.62164


### 2) SCAMP

In [35]:
scamp_kwargs_1 = {
    'config': {
        'ASTREF_CATALOG': 'USNO-A1',
        'ASTREF_BAND': 'DEFAULT',
        'SOLVE_PHOTOM': 'N',
        'CHECKPLOT_DEV': 'NULL'
        }
}
scamp_kwargs_1['code'] = 'SCAMP'
scamp_kwargs_1['config_file'] = os.path.join(paths['config'], 'default.scamp')

catalogs = 'temp/GRB130427_R60_001_001.cat'

In [36]:
scamp = aw.api.Astromatic(**scamp_kwargs_1)

In [37]:
this_cmd, kwargs2 = scamp.build_cmd(catalogs, **scamp_kwargs_1)
print this_cmd

scamp temp/GRB130427_R60_001_001.cat -c /home/ser/Dev/Notebooks/test_pipeline/config/default.scamp -ASTREF_BAND DEFAULT -SOLVE_PHOTOM N -CHECKPLOT_DEV NULL -ASTREF_CATALOG USNO-A1


In [38]:
scamp.run(catalogs)

{'status': 'success'}

In [48]:
# check header output after SCAMP work (need to load header from txt (.head) file)
fileobj = open('temp/GRB130427_R60_001_001.head')
hdr = fits.Header.fromtextfile(fileobj)
hdr

HISTORY   Astrometric solution by SCAMP version 2.0.4 (2014-07-09)              
COMMENT   (c) 2010-2013 IAP/CNRS/UPMC                                           
COMMENT                                                                         
EQUINOX =        2000.00000000 / Mean equinox                                   
RADESYS = 'ICRS    '           / Astrometric system                             
CTYPE1  = 'RA---TAN'           / WCS projection type for this axis              
CTYPE2  = 'DEC--TAN'           / WCS projection type for this axis              
CUNIT1  = 'DEG     '           / Axis unit                                      
CUNIT2  = 'DEG     '           / Axis unit                                      
CRVAL1  =   1.731463015344E+02 / World coordinate on this axis                  
CRVAL2  =   2.771173662660E+01 / World coordinate on this axis                  
CRPIX1  =   2.425000000000E+02 / Reference pixel on this axis                   
CRPIX2  =   2.865000000000E+

## 3) Swarp

In [52]:
files['image']
paths['temp']

'/home/ser/Dev/Notebooks/test_pipeline/temp'

In [67]:
coadd_fits_name = 'coadded.fits'
coadd_fits_filename = os.path.join(paths['temp'], coadd_fits_name)
swarp_kwargs_1 = {
    'config': {
        #'WEIGHT_TYPE': 'MAP_WEIGHT',
        #'WEIGHT_SUFFIX': '.wtmap.fits',
        'IMAGEOUT_NAME': coadd_fits_filename,
        'WEIGHTOUT_NAME': coadd_fits_filename.replace('.fits','.wtmap.fits'),
    }
    }

swarp_kwargs_1['code'] = 'SWarp'
swarp_kwargs_1['config_file'] = os.path.join(paths['config'], 'default.swarp')

In [68]:
swarp = aw.api.Astromatic(**swarp_kwargs_1)

In [69]:
this_cmd, kwargs2 = swarp.build_cmd('temp/GRB130427_R60_001_001.fit', **swarp_kwargs_1)
print this_cmd

swarp temp/GRB130427_R60_001_001.fit -c /home/ser/Dev/Notebooks/test_pipeline/config/default.swarp -WEIGHTOUT_NAME /home/ser/Dev/Notebooks/test_pipeline/temp/coadded.wtmap.fits -IMAGEOUT_NAME /home/ser/Dev/Notebooks/test_pipeline/temp/coadded.fits


In [70]:
swarp.run('temp/GRB130427_R60_001_001.fit')

{'status': 'success'}

### At now we have to check functions above on several images.

In [4]:
# function returns list of filenames with specified format in directory
def get_fits_images_from_dir(directory, files_format='fits', exact_pattern=True):
    regex_end = '$' if exact_pattern else ''
    files_format += regex_end
    return [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f)) and re.search(files_format, f, re.IGNORECASE) is not None]

In [5]:
# by defualt, we search for images in paths['images']
def pipeline_1(paths):
    print "prepare..."
    # prepare
    fits_format = 'fit'
    filenames = sorted(get_fits_images_from_dir(paths['images'], files_format=fits_format))
    exposures = []
    for filename in filenames:
        exposures.append({'image': os.path.join(paths['images'], filename)})
    # STEP 1: basic extraction with SExtractor (generate catalogs) for each image
    catalog_names = []
    print 'begin SExtractor execution'
    for files in exposures:
        print files['image'] + ':'
        # Create names for the output catalogs for each image (in temp dir)
        catalog_names.append(os.path.join(paths['temp'],
            os.path.basename(files['image']).replace('.'+fits_format, '.cat')))
        # set params for sextractor execution
        sex_kwargs_1 = {'code': 'SExtractor'}
        sex_kwargs_1['config_file'] = os.path.join(paths['config'], 'default.sex')
        sex_kwargs_1['config'] = {'CATALOG_NAME': catalog_names[-1]}
        sex_kwargs_1['config']['CATALOG_TYPE'] = 'FITS_LDAC'
        sex_kwargs_1['config']['FILTER'] = 'N'
        sex_kwargs_1['temp_path'] = paths['temp']
        sex_kwargs_1['params'] = ['NUMBER', 'EXT_NUMBER', 'XWIN_IMAGE', 'YWIN_IMAGE', 'AWIN_IMAGE', 'BWIN_IMAGE',
                                  'ERRAWIN_IMAGE','ERRBWIN_IMAGE', 'ERRTHETAWIN_IMAGE', 'ERRA_WORLD', 'ERRB_WORLD', 
                                  'ERRTHETA_WORLD', 'X_WORLD', 'Y_WORLD', 'XWIN_WORLD', 'YWIN_WORLD', 
                                  'FLUX_AUTO', 'FLUX_MAX', 'MAG_AUTO', 'FLUXERR_AUTO', 
                                  'FLAGS', 'FLUX_RADIUS', 'ELONGATION']
        sextractor = aw.api.Astromatic(**sex_kwargs_1)
        print sextractor.run(files['image'])
    print 'all catalogs successfully created'
    
    # STEP 2: compute solution for extracted catalogs
    print 'SCAMP now working...'
    scamp_kwargs_1 = {
        'config': {
            'ASTREF_CATALOG': 'USNO-B1',
            'ASTREF_BAND': 'DEFAULT',
            'SOLVE_PHOTOM': 'N',
            'CHECKPLOT_DEV': 'NULL'
            }
    }
    scamp_kwargs_1['code'] = 'SCAMP'
    scamp_kwargs_1['config_file'] = os.path.join(paths['config'], 'default.scamp')
    scamp = aw.api.Astromatic(**scamp_kwargs_1)
    this_cmd, kwargs2 = scamp.build_cmd(catalog_names, **scamp_kwargs_1)
    #print this_cmd
    print scamp.run(catalog_names)
    
    print 'SWarp now working...'
    # STEP 3: use SWarp for coadd images using .head files, computed with SCAMP
    coadd_fits_name = 'coadded.fits'
    coadd_fits_filename = os.path.join(paths['temp'], coadd_fits_name)
    swarp_kwargs_1 = {
        'config': {
            'IMAGEOUT_NAME': coadd_fits_filename,
            'WEIGHTOUT_NAME': coadd_fits_filename.replace('.fits','.wtmap.fits'),
        }
        }
    swarp_kwargs_1['code'] = 'SWarp'
    swarp_kwargs_1['config_file'] = os.path.join(paths['config'], 'default.swarp')
    swarp = aw.api.Astromatic(**swarp_kwargs_1)
    this_cmd, kwargs2 = swarp.build_cmd([exp['image'] for exp in exposures], **swarp_kwargs_1)
    #print this_cmd
    print swarp.run([exp['image'] for exp in exposures])
    
    # STEP 4: extract object from coadded image using PSF
    print 'SExtractor now working...'
    coadd_weight_map = coadd_fits_filename.replace('.fits','.wtmap.fits')
    sex_kwargs_2 = {
            'config': {
                'CATALOG_NAME': coadd_fits_filename.replace('.fits', '.cat'),
                'CATALOG_TYPE': 'FITS_LDAC',
                'FILTER': 'N',
                'WEIGHT_IMAGE': coadd_weight_map,
                'WEIGHT_TYPE': 'MAP_WEIGHT',        
            },
            'params': ['NUMBER', 'EXT_NUMBER', 'XWIN_IMAGE', 'YWIN_IMAGE', 'ERRAWIN_IMAGE',
                'ERRBWIN_IMAGE', 'ERRTHETAWIN_IMAGE', 'XWIN_WORLD', 'YWIN_WORLD', 'FLUX_APER(1)',
                'FLUXERR_APER(1)', 'FLAGS', 'FLAGS_WEIGHT', 'FLUX_RADIUS',
                'ELONGATION', 'VIGNET(20,20)', 'SNR_WIN']
        }
    sex_kwargs_2['code'] = 'SExtractor'
    sex_kwargs_2['config_file'] = os.path.join(paths['config'], 'default.sex')
    sex_kwargs_2['temp_path'] = paths['temp']
    sextractor = aw.api.Astromatic(**sex_kwargs_2)
    print sextractor.run(coadd_fits_filename)
    
    # Calculate PSF
    print 'PSFex now working...'
    psf_kwargs_1 = {
        'config': {
            'CENTER_KEYS': 'XWIN_IMAGE,YWIN_IMAGE',
            'PSFVAR_KEYS': 'XWIN_IMAGE,YWIN_IMAGE',
            'CHECKPLOT_DEV': 'NULL',
            'PSF_SUFFIX': '.psf'
        }
    }
    psf_kwargs_1['code'] = 'PSFEx'
    psf_kwargs_1['config_file'] = os.path.join(paths['config'], 'default.psfex')
    psf_kwargs_1['temp_path'] = paths['temp']
    psfex = aw.api.Astromatic(**psf_kwargs_1)
    print psfex.run(coadd_fits_filename.replace('.fits', '.cat'))
    
    # Calculate PSF photometry for coadded image
    print 'SExtractor now working...'
    output_cat_name = 'result.ldac.fits'
    catalog_name = os.path.join(paths['catalogs'], output_cat_name)
    sex_kwargs_3 = {
            'config': {
                'PSF_NAME': os.path.join(coadd_fits_filename.replace('.fits', '.psf')),
                'CATALOG_TYPE': 'FITS_LDAC',
                'FILTER': 'N',
                'CATALOG_NAME': catalog_name,
                'WEIGHT_IMAGE': coadd_weight_map,
                'WEIGHT_TYPE': 'MAP_WEIGHT',
            },
            'params': ['NUMBER', 'EXT_NUMBER', 'XWIN_IMAGE', 'YWIN_IMAGE', 'ERRAWIN_IMAGE',
                'ERRBWIN_IMAGE', 'ERRTHETAWIN_IMAGE', 'XWIN_WORLD', 'YWIN_WORLD', 'FLUX_AUTO',
                'FLUXERR_AUTO', 'FLAGS', 'FLAGS_WEIGHT', 'FLUX_RADIUS',
                'ELONGATION', 'MAG_AUTO', 'MAGERR_AUTO', 'ALPHAPSF_SKY', 'DELTAPSF_SKY',
                'ERRX2PSF_WORLD','ERRY2PSF_WORLD', 'FLUX_PSF', 'FLUXERR_PSF', 'MAG_PSF', 'MAGERR_PSF']
        }
    sex_kwargs_3['code'] = 'SExtractor'
    sex_kwargs_3['config_file'] = os.path.join(paths['config'], 'default.sex')
    sex_kwargs_3['temp_path'] = paths['temp']
    sextractor = aw.api.Astromatic(**sex_kwargs_3)
    print sextractor.run(coadd_fits_filename)
    return catalog_name

In [6]:
catalog_name = pipeline_1(paths)
catalog = aw.utils.ldac.get_table_from_ldac(catalog_name)
catalog

prepare...
begin SExtractor execution
/home/ser/Dev/Notebooks/test_pipeline/images/GRB130427_R60_001_001.fit:
{'status': 'success'}
/home/ser/Dev/Notebooks/test_pipeline/images/GRB130427_R60_001_002.fit:
{'status': 'success'}
/home/ser/Dev/Notebooks/test_pipeline/images/GRB130427_R60_001_003.fit:
{'status': 'success'}
/home/ser/Dev/Notebooks/test_pipeline/images/GRB130427_R60_001_004.fit:
{'status': 'success'}
/home/ser/Dev/Notebooks/test_pipeline/images/GRB130427_R60_001_005.fit:
{'status': 'success'}
/home/ser/Dev/Notebooks/test_pipeline/images/GRB130427_R60_001_006.fit:
{'status': 'success'}
/home/ser/Dev/Notebooks/test_pipeline/images/GRB130427_R60_001_007.fit:
{'status': 'success'}
/home/ser/Dev/Notebooks/test_pipeline/images/GRB130427_R60_001_008.fit:
{'status': 'success'}
/home/ser/Dev/Notebooks/test_pipeline/images/GRB130427_R60_001_009.fit:
{'status': 'success'}
/home/ser/Dev/Notebooks/test_pipeline/images/GRB130427_R60_001_010.fit:
{'status': 'success'}
/home/ser/Dev/Notebook

NUMBER,EXT_NUMBER,XWIN_IMAGE,YWIN_IMAGE,ERRAWIN_IMAGE,ERRBWIN_IMAGE,ERRTHETAWIN_IMAGE,XWIN_WORLD,YWIN_WORLD,FLUX_AUTO,FLUXERR_AUTO,FLAGS,FLAGS_WEIGHT,FLUX_RADIUS,ELONGATION,MAG_AUTO,MAGERR_AUTO,ALPHAPSF_SKY,DELTAPSF_SKY,ERRX2PSF_WORLD,ERRY2PSF_WORLD,FLUX_PSF,FLUXERR_PSF,MAG_PSF,MAGERR_PSF
,,pix,pix,pix,pix,deg,deg,deg,ct,ct,,,pix,,mag,mag,deg,deg,deg2,deg2,ct,ct,mag,mag
int32,int16,float64,float64,float32,float32,float32,float64,float64,float32,float32,int16,int16,float32,float32,float32,float32,float64,float64,float64,float64,float32,float32,float32,float32
1,1,372.139765315,13.9131042692,0.246118,0.0810579,0.0,173.112397892,27.6315894054,0.0215645,79.4957,1,2,0.000376829,2.96744,4.16565,4003.45,173.112399019,27.6324125758,0.0,0.0,0.0,0.0,99.0,99.0
2,1,80.5226834819,16.5051499837,0.190472,0.190321,8.49262,173.200173618,27.6322720576,-60.0936,148.739,1,2,-1.46933,3.09904,99.0,99.0,173.200095443,27.6325611443,0.0,0.0,0.0,0.0,99.0,99.0
3,1,238.649095638,17.0097210864,0.12948,0.128456,1.78076,173.152578034,27.6324181076,342.465,153.963,1,2,1.04587,3.20376,-6.33654,0.488237,173.152932102,27.6325472654,0.0,0.0,0.0,0.0,99.0,99.0
4,1,409.920713343,17.0025961689,0.153586,0.147151,-0.865824,173.101025693,27.6324103336,169.333,79.5021,1,2,0.545886,1.43112,-5.57186,0.509877,173.10109003,27.6324627012,0.0,0.0,0.0,0.0,99.0,99.0
5,1,449.642235515,17.6985819239,0.0774448,0.0757072,3.22519,173.089069527,27.6325918443,838.763,119.301,3,2,0.790973,1.87213,-7.3091,0.154466,173.08910857,27.6325684817,0.0,0.0,0.0,0.0,99.0,99.0
6,1,452.017212783,17.4397426785,0.0662309,0.0450145,89.8933,173.088354698,27.632522544,669.006,236.878,19,2,0.645593,6.9231,-7.06358,0.384525,173.088270863,27.6324960947,0.0,0.0,0.0,0.0,99.0,99.0
7,1,476.445852091,17.7727485014,0.214055,0.181751,-1.30503,173.081001685,27.6326082862,254.729,97.371,1,2,0.681841,1.61503,-6.01519,0.415127,173.080897023,27.6326009597,0.0,0.0,0.0,0.0,99.0,99.0
8,1,64.0021260804,18.8787420843,0.112896,0.102502,-89.3676,173.205146617,27.632902875,206.363,79.5211,3,0,0.489473,1.62489,-5.78658,0.418485,173.205080308,27.6328312978,0.0,0.0,0.0,0.0,99.0,99.0


## Test pipeline components

In [29]:
def build_pipeline(pipeline, exposures, ref_catalog='IKI', ref_band='DEFAULT',
        stack_name = 'test_stack.fits', output_cat_name='test_psf.ldac.fits'):
    # Generate catalogs from sextractor
    catalog_names = []
    for files in exposures:
        # Create names for the output catalogs for each image
        catalog_names.append(os.path.join(pipeline.paths['temp'],
            os.path.basename(files['image']).replace('.fit', '.cat')))
        kwargs = {
            # image to SExtract
            'files': files,
            # Arguments to initialize Astromatic class
            'api_kwargs': {
                # Configuration parameters
                'config': {
                    'CATALOG_NAME': catalog_names[-1],
                    'CATALOG_TYPE': 'FITS_LDAC',
                    'FILTER': 'N',
                    #'WEIGHT_TYPE': 'MAP_WEIGHT',
                },
                # Output parameters
                'params': ['NUMBER', 'EXT_NUMBER', 'XWIN_IMAGE', 'YWIN_IMAGE', 'ERRAWIN_IMAGE',
                    'ERRBWIN_IMAGE', 'ERRTHETAWIN_IMAGE', 'XWIN_WORLD', 'YWIN_WORLD', 'FLUX_AUTO',
                    'FLUXERR_AUTO', 'IMAFLAGS_ISO', 'FLAGS', 'FLAGS_WEIGHT', 'FLUX_RADIUS',
                    'ELONGATION'],
            },
        }
        # Add the step to the pipeline
        pipeline.add_step(aw.api.run_sex, ['step1', 'SExtractor'], **kwargs)

    # Use SCAMP to get astrometric solutions
    kwargs = {
        'catalogs': catalog_names,
        'api_kwargs': {
            'config': {
                'ASTREF_CATALOG': ref_catalog,
                'ASTREF_BAND': ref_band,
                'SOLVE_PHOTOM': 'N',
                'CHECKPLOT_DEV': 'NULL'
            },
        }
    }
    pipeline.add_step(aw.api.run_scamp, ['step2', 'SCAMP'],**kwargs)

    # Resample (rotate and scale) and combine (stack) images using SWarp
    stack_filename = os.path.join(pipeline.paths['temp'], stack_name)
    kwargs = {
        'filenames': [exp['image'] for exp in exposures],
        'api_kwargs': {
            'config': {
                'WEIGHT_TYPE': 'MAP_WEIGHT',
                'WEIGHT_SUFFIX': '.wtmap.fits',
                'IMAGEOUT_NAME': stack_filename,
                'WEIGHTOUT_NAME': stack_filename.replace('.fits','.wtmap.fits'),
            },
        }
    }
    pipeline.add_step(aw.api.run_swarp, ['step3', 'SWarp'], **kwargs)

    # Get positions in stack for PSF photometry (SExtractor -> PSFex -> SExtractor)
    kwargs = {
        'files': {
            'image': stack_filename,
            'wtmap': stack_filename.replace('.fits', '.wtmap.fits')
        },
        'api_kwargs': {
            'config': {
                'CATALOG_TYPE': 'FITS_LDAC',
                'FILTER': 'N',
                'WEIGHT_TYPE': 'MAP_WEIGHT',
            },
            'params': ['NUMBER', 'EXT_NUMBER', 'XWIN_IMAGE', 'YWIN_IMAGE', 'ERRAWIN_IMAGE',
                'ERRBWIN_IMAGE', 'ERRTHETAWIN_IMAGE', 'XWIN_WORLD', 'YWIN_WORLD', 'FLUX_APER(1)',
                'FLUXERR_APER(1)', 'FLAGS', 'FLAGS_WEIGHT', 'FLUX_RADIUS',
                'ELONGATION', 'VIGNET(20,20)', 'SNR_WIN'],
        }
    }
    pipeline.add_step(aw.api.run_sex, ['step4', 'SExtractor'], **kwargs)

    # Calculate PSF
    kwargs = {
        'catalogs': stack_filename.replace('.fits', '.cat'),
        'api_kwargs': {
            'config': {
                'CENTER_KEYS': 'XWIN_IMAGE,YWIN_IMAGE',
                'PSFVAR_KEYS': 'XWIN_IMAGE,YWIN_IMAGE',
                'CHECKPLOT_DEV': 'NULL',
                'PSF_SUFFIX': '.psf'
            }
        }
    }
    pipeline.add_step(aw.api.run_psfex, ['step5', 'PSFEx'], **kwargs)

    # Calculate PSF photometry for coadded image
    catalog_name = os.path.join(pipeline.paths['catalogs'], output_cat_name)
    kwargs = {
        'files': {
            'image': stack_filename,
            'wtmap': stack_filename.replace('.fits', '.wtmap.fits')
        },
        'api_kwargs': {
            'config': {
                'PSF_NAME': os.path.join(stack_filename.replace('.fits', '.psf')),
                'CATALOG_TYPE': 'FITS_LDAC',
                'FILTER': 'N',
                'CATALOG_NAME': catalog_name,
                'WEIGHT_TYPE': 'MAP_WEIGHT',
            },
            'params': ['NUMBER', 'EXT_NUMBER', 'XWIN_IMAGE', 'YWIN_IMAGE', 'ERRAWIN_IMAGE',
                'ERRBWIN_IMAGE', 'ERRTHETAWIN_IMAGE', 'XWIN_WORLD', 'YWIN_WORLD', 'FLUX_AUTO',
                'FLUXERR_AUTO', 'FLAGS', 'FLAGS_WEIGHT', 'FLUX_RADIUS',
                'ELONGATION', 'MAG_AUTO', 'MAGERR_AUTO', 'ALPHAPSF_SKY', 'DELTAPSF_SKY',
                'ERRX2PSF_WORLD','ERRY2PSF_WORLD', 'FLUX_PSF', 'FLUXERR_PSF', 'MAG_PSF', 'MAGERR_PSF'],
        }
    }
    pipeline.add_step(aw.api.run_sex, ['step6', 'SExtractor'], **kwargs)

    def save_output(pipeline, old_stack, new_stack, old_cat, new_cat):
        # Copy the final stack and catalog from the temp folder
        import shutil
        # Move the weight map if it exists
        if os.path.isfile(old_stack.replace('.fits', '.wtmap.fits')):
            shutil.move(old_stack.replace('.fits', '.wtmap.fits'), new_stack.replace('.fits', '.wtmap.fits'))
        shutil.move(old_stack, new_stack)
        shutil.move(old_cat, new_cat)
        result = {
            'status': 'success'
        }
        return result

    kwargs = {
        'old_stack': stack_filename,
        'new_stack': os.path.join(pipeline.paths['stacks'], stack_name),
        'old_cat': catalog_name,
        'new_cat': os.path.join(pipeline.paths['catalogs'], output_cat_name)
    }
    pipeline.add_step(save_output, ['step7', 'save_output'], **kwargs)

    return pipeline